In [ ]:
import pandas as pd

# Load the data
house_sales_data = pd.read_csv("../own-project/house_sales.csv")

house_sales_data.head()

In [ ]:
# Task 1: Treat '--' as missing values in 'city' and replace with "Unknown"
# Replace '--' with NaN and count missing values
house_sales_data["city"] = house_sales_data["city"].replace("--", pd.NA)
missing_city = house_sales_data["city"].isna().sum()  # Count missing values in 'city'

# Replace missing values in 'city' with "Unknown"
house_sales_data["city"] = house_sales_data["city"].fillna("Unknown")

print("Missing values in 'city':", missing_city)